# Test spatial units in the modelling

This tests performance of different core spatial units (grids, OA, LSOA) in the modelling to figure out the best unit to be used internally. 

Units to compare:

- 2011 output areas (the data we have are linked to 2011 census, not 2021)
- square grid 100m (mimicking OSGB but not being aligned for simplicity of this exercise)
- H3 grid at resolution 9
- Enclosed tessellation cells

Targets to compare:

- Air pollution
- House price

Geographic locations to compare:

- Leeds (chunk 40)
- Newcastle (chunk 26)

The final models shall be trained on the England-wide data. This trains only on each AOI.

In [3]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import tobler
import libpysal
import numpy as np
from itertools import product
import esda

from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn import metrics
from tobler.util import h3fy

In [4]:
data_folder = "../../../demoland_data"

In [5]:
chunks = gpd.read_parquet(f"{data_folder}/raw/urban_morpho/local_auth_chunks.pq")

- get air
- for chunk
    - for geom
        - generate/load geoms
        - interpolate air
        - interpolate price
        - interpolate explanatory
        - save table
        - for weights
            - create weights
            - get lags
            - fit the model
            - save sklearn model
            - save demoland model
            - save some performance data

## Get Air Pollution data

In [6]:
pm10_21 = (
    pd.read_csv(
        "https://uk-air.defra.gov.uk/datastore/pcm/mappm102021g.csv",
        header=5,
        na_values=["MISSING"],
    )
    .set_index(["x", "y"])
    .drop(columns="gridcode")
    .to_xarray()
)
pm25_21 = (
    pd.read_csv(
        "https://uk-air.defra.gov.uk/datastore/pcm/mappm252021g.csv",
        header=5,
        na_values=["MISSING"],
    )
    .set_index(["x", "y"])
    .drop(columns="gridcode")
    .to_xarray()
)
no2_21 = (
    pd.read_csv(
        "https://uk-air.defra.gov.uk/datastore/pcm/mapno22021.csv",
        header=5,
        na_values=["MISSING"],
    )
    .set_index(["x", "y"])
    .drop(columns="gridcode")
    .to_xarray()
)
so2_21 = (
    pd.read_csv(
        "https://uk-air.defra.gov.uk/datastore/pcm/mapso22021.csv",
        header=5,
        na_values=["MISSING"],
    )
    .set_index(["x", "y"])
    .drop(columns="gridcode")
    .to_xarray()
)
pollutants_2021 = xr.merge([pm10_21, pm25_21, no2_21, so2_21])
aqi = (
    pollutants_2021.pm252021g
    + pollutants_2021.pm102021g / 2
    + pollutants_2021.no22021 / 4
    + pollutants_2021.so22021 / 10
)
pollutants_2021 = pollutants_2021.assign(aqi=aqi)

## Get House Price data

In [7]:
house_prices = gpd.read_parquet(f"{data_folder}/processed/house_prices/price_per_sqm_england.parquet")

## Get population data

In [8]:
pop = gpd.read_parquet(f"{data_folder}/processed/gb_population_estimates.pq")

## Get workplace data

In [9]:
wp = gpd.read_parquet(
    f"{data_folder}/raw/workplace_population/workplace_by_industry_gb.pq"
)

## Get CORINE data

In [10]:
corine = gpd.read_parquet(f"{data_folder}/raw/land_cover/corine_gb.pq")

In [11]:
corine_names = {
    "Code_18_124": "Land cover [Airports]",
    "Code_18_211": "Land cover [Non-irrigated arable land]",
    "Code_18_121": "Land cover [Industrial or commercial units]",
    "Code_18_421": "Land cover [Salt marshes]",
    "Code_18_522": "Land cover [Estuaries]",
    "Code_18_142": "Land cover [Sport and leisure facilities]",
    "Code_18_141": "Land cover [Green urban areas]",
    "Code_18_112": "Land cover [Discontinuous urban fabric]",
    "Code_18_231": "Land cover [Pastures]",
    "Code_18_311": "Land cover [Broad-leaved forest]",
    "Code_18_131": "Land cover [Mineral extraction sites]",
    "Code_18_123": "Land cover [Port areas]",
    "Code_18_122": "Land cover [Road and rail networks and associated land]",
    "Code_18_512": "Land cover [Water bodies]",
    "Code_18_243": "Land cover [Land principally occupied by agriculture, with significant areas of natural vegetation]",
    "Code_18_313": "Land cover [Mixed forest]",
    "Code_18_412": "Land cover [Peat bogs]",
    "Code_18_321": "Land cover [Natural grasslands]",
    "Code_18_322": "Land cover [Moors and heathland]",
    "Code_18_324": "Land cover [Transitional woodland-shrub]",
    "Code_18_111": "Land cover [Continuous urban fabric]",
    "Code_18_423": "Land cover [Intertidal flats]",
    "Code_18_523": "Land cover [Sea and ocean]",
    "Code_18_312": "Land cover [Coniferous forest]",
    "Code_18_133": "Land cover [Construction sites]",
    "Code_18_333": "Land cover [Sparsely vegetated areas]",
    "Code_18_332": "Land cover [Bare rocks]",
    "Code_18_411": "Land cover [Inland marshes]",
    "Code_18_132": "Land cover [Dump sites]",
    "Code_18_222": "Land cover [Fruit trees and berry plantations]",
    "Code_18_242": "Land cover [Complex cultivation patterns]",
    "Code_18_331": "Land cover [Beaches, dunes, sands]",
    "Code_18_511": "Land cover [Water courses]",
    "Code_18_334": "Land cover [Burnt areas]",
    "Code_18_244": "Land cover [Agro-forestry areas]",
    "Code_18_521": "Land cover [Coastal lagoons]",
}

Evaluation script

## Data

#### Prep data

In [12]:
opt_id = 26
opt = "newcastle"

chunk = chunks.loc[[opt_id]]

Prepare Air pollution data

In [13]:
bds = chunk.buffer(1000).total_bounds
pollutants_aoi = pollutants_2021.sel(
    x=slice(bds[0], bds[2]), y=slice(bds[1], bds[3])
)
pollutants_aoi_df = pollutants_aoi.to_dataframe().reset_index()
pollutants_aoi_df = gpd.GeoDataFrame(
    pollutants_aoi_df,
    geometry=gpd.points_from_xy(
        pollutants_aoi_df.x, pollutants_aoi_df.y, crs=27700
    ).buffer(500, cap_style=3),
)

Get CV splits

In [14]:
cv_ids = np.tile(np.arange(5), pollutants_aoi_df.shape[0]//5 + 1)[:pollutants_aoi_df.shape[0]]
rng = np.random.default_rng()
rng.shuffle(cv_ids)
pollutants_aoi_df["split"] = cv_ids

Prepare house price data

In [15]:
house_prices_aoi = house_prices.iloc[house_prices.sindex.query(chunk.geometry.item())]

Prepare population data

In [16]:
pop_aoi = pop[pop.code.isin(house_prices_aoi.code)]

Link population and house price (both are on OA).

In [17]:
pop_hp = house_prices_aoi.merge(pop_aoi[["code", "population"]], on="code")

 Prepare workplace pop data

In [18]:
wp_aoi = wp.iloc[wp.sindex.query(chunk.geometry.item())]

Prepare CORINE data

In [19]:
corine_aoi = corine.iloc[corine.sindex.query(chunk.geometry.item())]

Get morphometric data

In [20]:
data = gpd.read_parquet(f"{data_folder}/raw/urban_morpho/cells_{opt_id}.pq")

### H3
Create geometries

In [21]:
geoms = h3fy(chunk, resolution=9, buffer=False)

/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/pyproj/crs/crs.py:1293: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


Interpolate Air Quality

In [22]:
interp = tobler.area_weighted.area_interpolate(pollutants_aoi_df, geoms, intensive_variables=["aqi"])
geoms["air_quality_index"] = interp.aqi.values

/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: aqi, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")


 Interpolate OA data

In [23]:
interp_oa = tobler.area_weighted.area_interpolate(
    pop_hp,
    geoms,
    intensive_variables=["priceper"],
    extensive_variables=["population"],
)
geoms["house_price_index"] = interp_oa.priceper.values
geoms["population"] = interp_oa.population.values

Interpolate workplace population

In [24]:
wp_interpolated = tobler.area_weighted.area_interpolate(
    wp_aoi,
    geoms,
    extensive_variables=[
        "A, B, D, E. Agriculture, energy and water",
        "C. Manufacturing",
        "F. Construction",
        "G, I. Distribution, hotels and restaurants",
        "H, J. Transport and communication",
        "K, L, M, N. Financial, real estate, professional and administrative activities",
        "O,P,Q. Public administration, education and health",
        "R, S, T, U. Other",
    ],
)

geoms[
    [
        "A, B, D, E. Agriculture, energy and water",
        "C. Manufacturing",
        "F. Construction",
        "G, I. Distribution, hotels and restaurants",
        "H, J. Transport and communication",
        "K, L, M, N. Financial, real estate, professional and administrative activities",
        "O,P,Q. Public administration, education and health",
        "R, S, T, U. Other",
    ]
] = wp_interpolated.drop(columns="geometry").values

Interpolate CORINE

In [25]:
corine_interpolated = tobler.area_weighted.area_interpolate(
    corine_aoi, geoms, categorical_variables=["Code_18"]
)
corine_interpolated.columns = corine_interpolated.columns.map(corine_names)
interesting = [
    "Land cover [Discontinuous urban fabric]",
    "Land cover [Continuous urban fabric]",
    "Land cover [Non-irrigated arable land]",
    "Land cover [Industrial or commercial units]",
    "Land cover [Green urban areas]",
    "Land cover [Pastures]",
    "Land cover [Sport and leisure facilities]",
]
geoms[interesting] = corine_interpolated[interesting].values

Interpolate morphometrics

In [26]:
chars = data.columns.drop(
    [
        "hindex",
        "tessellation",
        "buildings",
        "nodeID",
        "edgeID_keys",
        "edgeID_values",
        "edgeID_primary",
        "sdbPer",
        "ssbElo",
        "stcOri",
        "sdcLAL",
        "mdcAre",
        "ltcAre",
        "ltcWRE",
        "mtdMDi",
        "lcdMes",
        "lddNDe",
        "sddAre",
        "mdsAre",
        "ldsAre",
        "lisCel",
        "ldePer",
        "lseCWA",
    ]
)
morhp_interpolated = tobler.area_weighted.area_interpolate(
    data, geoms, intensive_variables=chars.tolist()
)

geoms[morhp_interpolated.columns.drop("geometry")] = morhp_interpolated.drop(
    columns="geometry"
).values

/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbAre, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbCoA, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCCo, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCor, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbSqu, replacing with

Get split IDs

In [27]:
geoms_ix, poll_ix = pollutants_aoi_df.sindex.query(geoms.centroid, predicate="within", sort=True)

In [28]:
geoms["split"] = pollutants_aoi_df["split"].values[poll_ix]

Save the table

In [29]:
geoms.to_parquet(f"{data_folder}/spatial_units_test/tables/h3_{opt}.pq")

### square grid
Create geometries

In [30]:
coords = np.array(
    list(product(
        np.arange(bds[0], bds[2], 250),
        np.arange(bds[1], bds[3], 250),
    ))
)
points = gpd.GeoSeries.from_xy(coords[:, 0], coords[:, 1], crs=27700)
geoms = points.iloc[points.sindex.query(chunk.geometry.item(), predicate="contains")].buffer(100, cap_style=3).to_frame("geometry")

Interpolate Air Quality

In [31]:
interp = tobler.area_weighted.area_interpolate(pollutants_aoi_df, geoms, intensive_variables=["aqi"])
geoms["air_quality_index"] = interp.aqi.values

/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: aqi, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")


 Interpolate OA data

In [32]:
interp_oa = tobler.area_weighted.area_interpolate(
    pop_hp,
    geoms,
    intensive_variables=["priceper"],
    extensive_variables=["population"],
)
geoms["house_price_index"] = interp_oa.priceper.values
geoms["population"] = interp_oa.population.values

Interpolate workplace population

In [33]:
wp_interpolated = tobler.area_weighted.area_interpolate(
    wp_aoi,
    geoms,
    extensive_variables=[
        "A, B, D, E. Agriculture, energy and water",
        "C. Manufacturing",
        "F. Construction",
        "G, I. Distribution, hotels and restaurants",
        "H, J. Transport and communication",
        "K, L, M, N. Financial, real estate, professional and administrative activities",
        "O,P,Q. Public administration, education and health",
        "R, S, T, U. Other",
    ],
)

geoms[
    [
        "A, B, D, E. Agriculture, energy and water",
        "C. Manufacturing",
        "F. Construction",
        "G, I. Distribution, hotels and restaurants",
        "H, J. Transport and communication",
        "K, L, M, N. Financial, real estate, professional and administrative activities",
        "O,P,Q. Public administration, education and health",
        "R, S, T, U. Other",
    ]
] = wp_interpolated.drop(columns="geometry").values

Interpolate CORINE

In [34]:
corine_interpolated = tobler.area_weighted.area_interpolate(
    corine_aoi, geoms, categorical_variables=["Code_18"]
)
corine_interpolated.columns = corine_interpolated.columns.map(corine_names)
interesting = [
    "Land cover [Discontinuous urban fabric]",
    "Land cover [Continuous urban fabric]",
    "Land cover [Non-irrigated arable land]",
    "Land cover [Industrial or commercial units]",
    "Land cover [Green urban areas]",
    "Land cover [Pastures]",
    "Land cover [Sport and leisure facilities]",
]
geoms[interesting] = corine_interpolated[interesting].values

Interpolate morphometrics

In [35]:
chars = data.columns.drop(
    [
        "hindex",
        "tessellation",
        "buildings",
        "nodeID",
        "edgeID_keys",
        "edgeID_values",
        "edgeID_primary",
        "sdbPer",
        "ssbElo",
        "stcOri",
        "sdcLAL",
        "mdcAre",
        "ltcAre",
        "ltcWRE",
        "mtdMDi",
        "lcdMes",
        "lddNDe",
        "sddAre",
        "mdsAre",
        "ldsAre",
        "lisCel",
        "ldePer",
        "lseCWA",
    ]
)
morhp_interpolated = tobler.area_weighted.area_interpolate(
    data, geoms, intensive_variables=chars.tolist()
)

geoms[morhp_interpolated.columns.drop("geometry")] = morhp_interpolated.drop(
    columns="geometry"
).values

/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbAre, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbCoA, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCCo, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCor, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbSqu, replacing with

Get split IDs

In [36]:
geoms_ix, poll_ix = pollutants_aoi_df.sindex.query(geoms.centroid, predicate="within", sort=True)

In [37]:
geoms["split"] = pollutants_aoi_df["split"].values[poll_ix]

Save the table

In [38]:
geoms.to_parquet(f"{data_folder}/spatial_units_test/tables/square_{opt}.pq")

### output area
Create geometries

In [39]:
geoms = pop_hp.set_index("code")[["geometry", "priceper", "population"]].set_crs(27700, allow_override=True)

Interpolate Air Quality

In [40]:
interp = tobler.area_weighted.area_interpolate(pollutants_aoi_df, geoms, intensive_variables=["aqi"])
geoms["air_quality_index"] = interp.aqi.values

/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: aqi, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")


 Rename OA data

In [41]:
geoms = geoms.rename(columns={"priceper": "house_price_index"})

Interpolate workplace population

In [42]:
wp_interpolated = tobler.area_weighted.area_interpolate(
    wp_aoi,
    geoms,
    extensive_variables=[
        "A, B, D, E. Agriculture, energy and water",
        "C. Manufacturing",
        "F. Construction",
        "G, I. Distribution, hotels and restaurants",
        "H, J. Transport and communication",
        "K, L, M, N. Financial, real estate, professional and administrative activities",
        "O,P,Q. Public administration, education and health",
        "R, S, T, U. Other",
    ],
)

geoms[
    [
        "A, B, D, E. Agriculture, energy and water",
        "C. Manufacturing",
        "F. Construction",
        "G, I. Distribution, hotels and restaurants",
        "H, J. Transport and communication",
        "K, L, M, N. Financial, real estate, professional and administrative activities",
        "O,P,Q. Public administration, education and health",
        "R, S, T, U. Other",
    ]
] = wp_interpolated.drop(columns="geometry").values

Interpolate CORINE

In [43]:
corine_interpolated = tobler.area_weighted.area_interpolate(
    corine_aoi, geoms, categorical_variables=["Code_18"]
)
corine_interpolated.columns = corine_interpolated.columns.map(corine_names)
interesting = [
    "Land cover [Discontinuous urban fabric]",
    "Land cover [Continuous urban fabric]",
    "Land cover [Non-irrigated arable land]",
    "Land cover [Industrial or commercial units]",
    "Land cover [Green urban areas]",
    "Land cover [Pastures]",
    "Land cover [Sport and leisure facilities]",
]
geoms[interesting] = corine_interpolated[interesting].values

Interpolate morphometrics

In [44]:
chars = data.columns.drop(
    [
        "hindex",
        "tessellation",
        "buildings",
        "nodeID",
        "edgeID_keys",
        "edgeID_values",
        "edgeID_primary",
        "sdbPer",
        "ssbElo",
        "stcOri",
        "sdcLAL",
        "mdcAre",
        "ltcAre",
        "ltcWRE",
        "mtdMDi",
        "lcdMes",
        "lddNDe",
        "sddAre",
        "mdsAre",
        "ldsAre",
        "lisCel",
        "ldePer",
        "lseCWA",
    ]
)
morhp_interpolated = tobler.area_weighted.area_interpolate(
    data, geoms, intensive_variables=chars.tolist()
)

geoms[morhp_interpolated.columns.drop("geometry")] = morhp_interpolated.drop(
    columns="geometry"
).values

/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbAre, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbCoA, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCCo, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCor, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbSqu, replacing with

Get split IDs

In [45]:
geoms_ix, poll_ix = pollutants_aoi_df.sindex.query(geoms.centroid, predicate="within", sort=True)

In [46]:
geoms.loc[geoms.index[geoms_ix], "split"] = pollutants_aoi_df["split"].values[poll_ix]

Save the table

In [47]:
geoms.to_parquet(f"{data_folder}/spatial_units_test/tables/oa_{opt}.pq")

### ET cells
Create geometries

In [48]:
geoms = data.set_index("hindex").drop(columns=
    [
        "buildings",
        "nodeID",
        "edgeID_keys",
        "edgeID_values",
        "edgeID_primary",
        "sdbPer",
        "ssbElo",
        "stcOri",
        "sdcLAL",
        "mdcAre",
        "ltcAre",
        "ltcWRE",
        "mtdMDi",
        "lcdMes",
        "lddNDe",
        "sddAre",
        "mdsAre",
        "ldsAre",
        "lisCel",
        "ldePer",
        "lseCWA",
    ]
)

Interpolate Air Quality

In [49]:
interp = tobler.area_weighted.area_interpolate(pollutants_aoi_df, geoms, intensive_variables=["aqi"])
geoms["air_quality_index"] = interp.aqi.values

/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: aqi, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")


 Interpolate OA data

In [50]:
interp_oa = tobler.area_weighted.area_interpolate(
    pop_hp,
    geoms,
    intensive_variables=["priceper"],
    extensive_variables=["population"],
)
geoms["house_price_index"] = interp_oa.priceper.values
geoms["population"] = interp_oa.population.values

Interpolate workplace population

In [51]:
wp_interpolated = tobler.area_weighted.area_interpolate(
    wp_aoi,
    geoms,
    extensive_variables=[
        "A, B, D, E. Agriculture, energy and water",
        "C. Manufacturing",
        "F. Construction",
        "G, I. Distribution, hotels and restaurants",
        "H, J. Transport and communication",
        "K, L, M, N. Financial, real estate, professional and administrative activities",
        "O,P,Q. Public administration, education and health",
        "R, S, T, U. Other",
    ],
)

geoms[
    [
        "A, B, D, E. Agriculture, energy and water",
        "C. Manufacturing",
        "F. Construction",
        "G, I. Distribution, hotels and restaurants",
        "H, J. Transport and communication",
        "K, L, M, N. Financial, real estate, professional and administrative activities",
        "O,P,Q. Public administration, education and health",
        "R, S, T, U. Other",
    ]
] = wp_interpolated.drop(columns="geometry").values

Interpolate CORINE

In [52]:
corine_interpolated = tobler.area_weighted.area_interpolate(
    corine_aoi, geoms, categorical_variables=["Code_18"]
)
corine_interpolated.columns = corine_interpolated.columns.map(corine_names)
interesting = [
    "Land cover [Discontinuous urban fabric]",
    "Land cover [Continuous urban fabric]",
    "Land cover [Non-irrigated arable land]",
    "Land cover [Industrial or commercial units]",
    "Land cover [Green urban areas]",
    "Land cover [Pastures]",
    "Land cover [Sport and leisure facilities]",
]
geoms[interesting] = corine_interpolated[interesting].values

Get split IDs

In [53]:
geoms_ix, poll_ix = pollutants_aoi_df.sindex.query(geoms.centroid, predicate="within", sort=True)

In [54]:
geoms.loc[geoms.index[geoms_ix], "split"] = pollutants_aoi_df["split"].values[poll_ix]

Save the table

In [55]:
geoms.to_parquet(f"{data_folder}/spatial_units_test/tables/et_{opt}.pq")